In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
import os
os.chdir(r'/workspaces/demo-repo/src/')

import prepare_data, train, preprocess

warnings.filterwarnings('ignore')
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv("/workspaces/demo-repo/data/breast-cancer-wisconsin-data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,"1,001.00",0.12,0.28,0.30,0.15,0.24,0.08,1.09,0.91,8.59,153.40,0.01,0.05,0.05,0.02,0.03,0.01,25.38,17.33,184.60,"2,019.00",0.16,0.67,0.71,0.27,0.46,0.12,NaN
1,842517,M,20.57,17.77,132.90,"1,326.00",0.08,0.08,0.09,0.07,0.18,0.06,0.54,0.73,3.40,74.08,0.01,0.01,0.02,0.01,0.01,0.00,24.99,23.41,158.80,"1,956.00",0.12,0.19,0.24,0.19,0.28,0.09,NaN
2,84300903,M,19.69,21.25,130.00,"1,203.00",0.11,0.16,0.20,0.13,0.21,0.06,0.75,0.79,4.58,94.03,0.01,0.04,0.04,0.02,0.02,0.00,23.57,25.53,152.50,"1,709.00",0.14,0.42,0.45,0.24,0.36,0.09,NaN
3,84348301,M,11.42,20.38,77.58,386.10,0.14,0.28,0.24,0.11,0.26,0.10,0.50,1.16,3.44,27.23,0.01,0.07,0.06,0.02,0.06,0.01,14.91,26.50,98.87,567.70,0.21,0.87,0.69,0.26,0.66,0.17,NaN
4,84358402,M,20.29,14.34,135.10,"1,297.00",0.10,0.13,0.20,0.10,0.18,0.06,0.76,0.78,5.44,94.44,0.01,0.02,0.06,0.02,0.02,0.01,22.54,16.67,152.20,"1,575.00",0.14,0.20,0.40,0.16,0.24,0.08,NaN


In [5]:
config = prepare_data.load_config()

2023-08-24 09:47:56,342 - DEBUG - loading config.yml...


In [7]:
model = 'RandomForestClassifier'
ds_config = config['diabetes']

In [8]:
ds_config

{'name': 'diabetes',
 'input_file': {'name': 'diabetes.csv',
  'path': '/workspaces/demo-repo/data',
  'drop_nulls': ['outcome']},
 'target': 'outcome',
 'models': {'LogisticRegression': 'none', 'RandomForestClassifier': 'none'}}

In [9]:
if 'params' in ds_config['models']['RandomForestClassifier']:
    print(ds_config['models']['RandomForestClassifier']['params'])

In [11]:
df = prepare_data.prepare(model, ds_config)
df.head()

2023-08-24 09:49:06,038 - DEBUG - loading dataset diabetes.csv...
2023-08-24 09:49:06,074 - DEBUG - convert all column names to lowercase...
2023-08-24 09:49:06,077 - DEBUG - dropping NaN from ['outcome']...


,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,outcome
0,6,148,72,35,0,33.60,0.63,50,1
1,1,85,66,29,0,26.60,0.35,31,0
2,8,183,64,0,0,23.30,0.67,32,1
3,1,89,66,23,94,28.10,0.17,21,0
4,0,137,40,35,168,43.10,2.29,33,1


In [14]:
if 'transform' in ds_config:
    df = preprocess.apply_preprocessing(df, model, ds_config)

In [16]:
train.train_and_predict(df, model, ds_config)

2023-08-24 09:51:36,495 - DEBUG - loading prepared data for training...
2023-08-24 09:51:36,498 - DEBUG - splitting train and test...
2023-08-24 09:51:36,940 - DEBUG - accuracy for RandomForestClassifier(): 0.8072916666666666
